<a href="https://colab.research.google.com/github/mohanvamsitrade/Gen_AI_Projects/blob/main/Restaurant_Advisor_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Firstly Importing and Installing few Libraries

1. Obtain a API key from Huggingface/OpenAI (Had some issues with OPENAI API, So continuing with Huggingface)
2. Installing Langchain Framework
3. Installing tiktoken and cohere (You may need these incase of using Hugginface, If using OpenAI model you may ignore unless OPENAI installation throws any error)
4. Install Huggingface or OPENAI depending on the LLM model which you are using


In [ ]:
from secret_key import huggingface_key1

import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingface_key1

In [ ]:
!pip install langchain

In [ ]:
!pip install tiktoken

In [ ]:
!pip install cohere


In [ ]:
pip install --upgrade typing-extensions==4.7

In [ ]:
!pip install --upgrade llmx

In [ ]:
!pip install huggingface

In [ ]:
!pip -q install langchain huggingface_hub transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.2 MB/s eta 0:00:00


Langchain is key framework needed which working with LLM's.. Its lot easier with Lanchain trust me

1. Import HuggingFaceHub from LangChain
2. Initiating HuggingFaceHub into a variable llm
3. I have considered a model "google/flan-t5-xxl" from huggingface, you can use other models in (Seq2Seq).. For now lets go with this model..
4. Incase of having OpenAI API key, you can continiue with OPENAI model..As per my comparision for the below program OPENAI works much better than the FLAN-T5 which i have considered

In [ ]:
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id = "google/flan-t5-xxl", model_kwargs= {'temperature':0.6})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


The above few steps are good enough to start using the model which we selected

1. A simple prompt given into the model yields result just like Chat GPT

In [ ]:
name = llm("I want to open a restaurant for Indian food. Suggest a fency name for this")
print(name)

menu = llm("Give me a 5 menu names for Indian restaurant")
print(menu)

Indian Fusion
tandoori chicken tikka nan sag aloo dhaniya korma saag paneer


Prompt Templates are the next important modules in Langchain which help you in giving multiple prompts without repeting much of the sentences

1. Below i used PromptTemplate to customize a prompt with a certain input variable of my interest so that i can make changes just to the variable and acquire different results while using the model
2. Below i a trying to extract some fancy restaurant names from the model based on the cuisine of my interest
3. I have also created another prompt which queiries my model for some menu items based on the restaurant name give

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name  = PromptTemplate(
    input_variables=['cusine'], template = "I want to open a restaurant for {cusine} food. Suggest a fency name for this"
)

prompt_template_name.format(cusine = 'English')

prompt_template_menu = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest 6 menu items for {restaurant_name}. Return it as a comma separated list"
)

LLMChains help you in using both the model and the Prompttemplate under a signal umberella

1. Below i have created "name_chain" and a "menu_chain"

In [ ]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm = llm, prompt= prompt_template_name)
menu_chain = LLMChain(llm = llm, prompt= prompt_template_menu)
name_chain.invoke('Indian')
menu_chain.invoke('Indian')

{'restaurant_name': 'Indian', 'text': ',,,, '}

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, menu_chain])

response = chain.invoke("Mexican")
print(response)



{'input': 'Mexican', 'output': 'fajitas, enchiladas, burrito, carnitas, chile relleno'}


Now lets use all the knowledge we have gained earlier

1. First creating 2 prompts which we provide "Cusine" and "resturant_name" as variables to the prompts individually
2. creating LLMChain's using the prompts, we will be using the same LLM model "google/flan-t5-xxl" as earlier.. No changes to the model made

In [ ]:
prompt_template_name  = PromptTemplate(
    input_variables=['cusine'],
    template = "I want to open a restaurant for {cusine} food. Suggest a fency name for this"
)

prompt_template_menu = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest 6 menu items for {restaurant_name}. Return it as a comma separated list"
)

name_chain = LLMChain(llm = llm, prompt= prompt_template_name,output_key="restaurant_name")
menu_chain = LLMChain(llm = llm, prompt= prompt_template_menu, output_key= "menu_items")

Now we are upgrading ourselves here.. We are going to link the 2 LLMChian's which we created earlier

Simply to say, the output of the first LLMChain acts as the input for the 2nd LLMChain

1. Lets import SequentialChain from Langchain.chains (You might have already understood the significance of SequentialChain module here.. Using LLMChains in Sequentially)
2. So our first LLM Chain is "name_chain" which takes input "Cuisine" and gives output "restautant_name"
3. Now the input for the sencond LLM Chain is "restaurant_name" and gives "menu_items" as output
4. Using using both chains in our SequentialChain module gives considers "cuisine" as input variable and provides output of "restaurant_name" and "menu_items" as output variable

Great Work guys.. If you have come till here.. You have made your first step in using GENAI as per your interest

Now lets create a short app using streamlit using this simple model

In [ ]:
from langchain.chains import SequentialChain

new_chain = SequentialChain(
    chains = [name_chain, menu_chain],
    input_variables=['cusine'],
    output_variables= ['restaurant_name','menu_items']
)

new_chain({'cusine':'Mexican'})

{'cusine': 'Mexican',
 'restaurant_name': 'Mexican food restaurant',
 'menu_items': 'fajitas, enchiladas, burrito, carnitas, chile relleno'}